## Подготовка данных и разведовательный анализ

In [1]:
# imports
import pandas as pd
import pandas_profiling
from functools import reduce


Bad key "text.kerning_factor" on line 4 in
C:\Users\MikhailS\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


#### Константы
- пути к данным
- признаки, описывающие данные
- типы "столбцов", или "подпризнаков"
- ID ТС

In [2]:
DATA_PATH = 'data/vehicle_dataset_public/'
DATA_PROC_PATH = 'data/vehicle_proccessed/'

TAGS_FEATURES = ['fuelLevel', 'ingection', 'speedAndHeight', 'tachometer']
TAGS_TARGET = ['refueling2']
TAGS = TAGS_FEATURES + TAGS_TARGET

TIME_COLUMNS = ['DTIME', 'STARTDATE', 'ENDDATE']
FLOAT_COLUMNS = ['BEVALUE', 'SPEED', 'HEIGHT']

VEHICLE_IDS = [1, 3, 5, 19, 28]

In [3]:
# Словарь, состоящий из dataframe'ов для каждого признака
def dict_df_vehicle_i(i):
    dict_df_vehicle = {}
    for tag in TAGS:
        df = pd.read_csv(DATA_PATH +
                         f'vehicle{i}_{tag}_public.csv', sep=';')
        for col in FLOAT_COLUMNS:
            if col in df.columns and df[col].dtype == 'object':
                df[col] = df[col].str.replace(',','.').astype(float)
        df.rename(columns={'BEVALUE': tag}, inplace=True)

        for time_col in TIME_COLUMNS:
            if time_col in df.columns:
                df[time_col] = pd.to_datetime(df[time_col])

        dict_df_vehicle[tag] = df

    return dict_df_vehicle

In [4]:
# Объединение всех "исходных" признаков из словаря dataframe'ов
# в один dataframe
def df_summary(dict_df):
    df_to_merge = [dict_df[name] for name in TAGS_FEATURES]

    df_res = reduce(lambda left, right: pd.merge(left, right, how='outer', on='DTIME'),
                    df_to_merge)
    df_res.columns = df_res.columns.str.lower()

    return df_res

In [5]:
def combine_data():
    for i in VEHICLE_IDS:
        dict_df = dict_df_vehicle_i(i)
        df_sum = df_summary(dict_df)

        report = pandas_profiling.ProfileReport(df_sum)

        name = f'vehicle{i}_'
        df_sum.to_csv(DATA_PROC_PATH + name + 'summary.csv', index_label='i')
        dict_df['refueling2'].to_csv(DATA_PROC_PATH + name + 'fueling2.csv', index_label='i')
        report.to_file(DATA_PROC_PATH + 'reports/' + name + 'report.html')

In [6]:
combine_data()

In [7]:
def df_for_all_vehicles():
    df_sum_list = []
    df_refuel_list = []
    for i in VEHICLE_IDS:
        dict_df = dict_df_vehicle_i(i)
        df_sum = df_summary(dict_df)
        df_sum['TSID'] = str(i)

        df_sum_list.append(df_sum)
        df_refuel_list.append(dict_df['refueling2'])

    df_sum_all = reduce(lambda left, right:
                    pd.concat([left, right], ignore_index=True), df_sum_list)
    df_refuel_all = reduce(lambda left, right:
                    pd.concat([left, right], ignore_index=True), df_refuel_list)

    return df_sum_all, df_refuel_all

In [8]:
df_sum_all, df_refuel_all = df_for_all_vehicles()

df_sum_all

,dtime,fuellevel,ingection,speed,height,tachometer,TSID
0,2020-01-08 21:16:02,0.0,0,0,-22.9,0,1
1,2020-01-08 21:17:04,0.0,0,0,-22.9,0,1
2,2020-01-08 21:18:04,0.0,0,0,-22.9,0,1
3,2020-01-09 10:05:26,49.7,1,0,-22.9,1248,1
4,2020-01-09 10:06:27,49.9,1,0,-22.9,1056,1
...,...,...,...,...,...,...,...
290141,2020-06-30 18:13:27,NaN,1,16,255.4,1792,28
290142,2020-06-30 18:13:52,NaN,1,39,250.6,2368,28
290143,2020-06-30 18:14:25,NaN,1,12,244.4,2400,28
290144,2020-06-30 18:14:33,NaN,1,5,243.4,1888,28


In [9]:
df_refuel_all

,TSID,STARTDATE,ENDDATE,STARTLEVEL,ENDLEVEL
0,1,2020-01-16 11:02:40,2020-01-16 13:20:40,"46,7","38,4"
1,1,2020-01-16 13:20:40,2020-01-16 13:25:46,"38,4","56,7"
2,1,2020-01-17 09:05:49,2020-01-17 12:14:10,"56,3",51
3,1,2020-01-17 14:45:48,2020-01-17 16:12:27,51,"43,9"
4,1,2020-01-20 09:44:03,2020-01-20 12:44:07,"43,9","23,5"
...,...,...,...,...,...
358,28,2020-06-29 07:45:02,2020-06-29 09:30:02,27,19.9
359,28,2020-06-29 20:52:02,2020-06-29 20:58:02,9.3,58.9
360,28,2020-06-29 20:58:02,2020-06-29 21:59:02,58.9,53.7
361,28,2020-06-30 07:49:13,2020-06-30 09:39:13,53.7,46.4


In [10]:
df_sum_all.to_csv(DATA_PROC_PATH + 'all_summary.csv', index_label='i')
df_refuel_all.to_csv(DATA_PROC_PATH + 'all_refuel.csv', index_label='i')

In [11]:
reportSumData = pandas_profiling.ProfileReport(df_sum_all)
reportRefuel = pandas_profiling.ProfileReport(df_refuel_all)

In [12]:
reportSumData.to_file(DATA_PROC_PATH + 'reports/all_summary_report.html')
reportRefuel.to_file(DATA_PROC_PATH + 'reports/all_refuel_report.html')